# Poggi's Conference Acceptance Rate Data Integration

Jupyter Notebook for the processing and integration of the Conference Acceptance Rate Data obtained by Prof. Francesco Poggi, professor at the University of Modena and Reggio Emilia.

*The CORE Conference Ranking provides assessments of major conferences in the computing disciplines.The rankings are managed by the CORE Executive Committee, with periodic rounds for submission of requests for addition or reranking of conferences. Decisions are made by academic committees based on objective data requested as part of the submission process.* (source: CORE)

The data was obtained by web scraping and is provided in JSON format.
____________________________________________________________

For this process, the following CSV files are needed: ```out_citations_and_conferences_location_ready_v2.csv``` and the Conference Acceptance Rate JSON files. 

The first one must be generated running the Notebook ```1 - Citation and Locations Dataset Preparation.ipynb``` that is contained in the ```5 - Conference Locations Ranking Integration``` folder of this Repository.<br>
Poggi's Conference Acceptance Rate JSON files can be downloaded from [here]().

In particular, the following operations are going to be executed:
* Opening of the CSV conference citations and locations dataset
* Extraction of the distinct conference series name from the conference citations and locations dataset
* (Sequential) Reading of the CORE CSV files
* Drop of the useless CORE columns
* Filter of the conferences without a rank
* Join between the distinct conference series name and the CORE ratings

Lastly, the processed dataset is going to be saved on disk in CSV format

In [1]:
# Libraries Import
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [2]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/data_acceptance/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

## Read and Preparation of the Citation Dataset

In [3]:
df_citations_and_locations = pd.read_csv(path_file_export + 'out_citations_and_conferences_location_ready_v2.csv', low_memory=False, index_col=[0])
print(f'Successfully Imported the Conference Citations and Locations Ready CSV')

Successfully Imported the Conference Citations and Locations Ready CSV


In [4]:
df_citations_and_locations.head(3)

,CitationCount_COCI,CitationCount_Mag,CitationCount_MagEstimated,ConferenceLocation,ConferenceNormalizedName,ConferenceSeriesNormalizedName,Doi,Year
0,10,12,12,"Austin, Texas, United States",disc 2014,disc,10.1007/978-3-662-45174-8_28,2014
1,5,10,10,"Wrocław, Lower Silesian Voivodeship, Poland",esa 2014,esa,10.1007/978-3-662-44777-2_60,2014
2,11,20,20,"Innsbruck, Tyrol, Austria",enter 2013,enter,10.1007/978-3-319-03973-2_13,2013


### Extracion of the Distinct Conference Series from the Conference and Locations Datasets

In [5]:
df_conference_series = df_citations_and_locations.drop_duplicates(subset="ConferenceSeriesNormalizedName")

#filter of the useless columns
df_conference_series = df_conference_series.drop(df_conference_series.columns.difference(["ConferenceSeriesNormalizedName"]), axis=1)

# drop of the nan row
df_conference_series = df_conference_series.dropna(subset={"ConferenceSeriesNormalizedName"})

# reset of the index
df_conference_series = df_conference_series.reset_index(drop=True)

df_conference_series

,ConferenceSeriesNormalizedName
0,disc
1,esa
2,enter
3,dexa
4,icaisc
...,...
5307,infinity
5308,calculemus
5309,agp
5310,sci


## Read and Processing of the AcmConferences JSON

In [6]:
df_AcmConferences_raw = pd.read_json(path_file_import + 'AcmConferences.json')
print(f'Successfully Imported theAcmConferences JSON')

Successfully Imported theAcmConferences JSON


Print of the dataset structure:

In [15]:
df_AcmConferences_raw.head(3)

,confName,confAcronym,confUrl,info
0,DocEng: Document Engineering,DocEng,https://dl.acm.org/action/doSearch?target=brow...,"[{'year': '19', 'yearUnparsed': 'DocEng '19', ..."
1,"MANPU: Comics Aanalysis, Processing and Unders...",MANPU,https://dl.acm.org/action/doSearch?target=brow...,"[{'year': '16', 'yearUnparsed': 'MANPU '16', '..."
2,Nanoarch: Nanoscale Architectures,Nanoarch,https://dl.acm.org/action/doSearch?target=brow...,"[{'year': '18', 'yearUnparsed': 'NANOARCH '18'..."


### Expansion of the info field

In [23]:
df_Acm_first_level = pd.json_normalize(df_AcmConferences_raw.iloc[[0]]['info'])
df_Acm_first_level

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,"{'year': '19', 'yearUnparsed': 'DocEng '19', '...","{'year': '17', 'yearUnparsed': 'DocEng '17', '...","{'year': '16', 'yearUnparsed': 'DocEng '16', '...","{'year': '15', 'yearUnparsed': 'DocEng '15', '...","{'year': '14', 'yearUnparsed': 'DocEng '14', '...","{'year': '13', 'yearUnparsed': 'DocEng '13', '...","{'year': '12', 'yearUnparsed': 'DocEng '12', '...","{'year': '11', 'yearUnparsed': 'DocEng '11', '...","{'year': '10', 'yearUnparsed': 'DocEng '10', '...","{'year': '09', 'yearUnparsed': 'DocEng '09', '...","{'year': '08', 'yearUnparsed': 'DocEng '08', '...","{'year': '07', 'yearUnparsed': 'DocEng '07', '...","{'year': '06', 'yearUnparsed': 'DocEng '06', '...","{'year': '05', 'yearUnparsed': 'DocEng '05', '...","{'year': '04', 'yearUnparsed': 'DocEng '04', '...","{'year': '03', 'yearUnparsed': 'DocEng '03', '...","{'year': '02', 'yearUnparsed': 'DocEng '02', '...","{'year': '01', 'yearUnparsed': 'DocEng '01', '..."


In [12]:
pd.json_normalize(df_Acm_first_level.iloc[:, 0])

,year,yearUnparsed,yearUrl,submitted,accepted,percAccepted
0,19,DocEng '19,https://dl.acm.org/doi/proceedings/10.1145/334...,77,30,39%


In [46]:
for df_second_level in df_Acm_first_level:

    for items in df_Acm_first_level[df_second_level].items():
        print(type(items[1]))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


### Dataset Processing

In [9]:
df_AcmConferences = pd.DataFrame()
df_AcmConferences.Conf_Acronym = np.nan
df_AcmConferences.Year = np.nan
df_AcmConferences.Papers_Submitted = np.nan
df_AcmConferences.Papers_Accepted = np.nan
df_AcmConferences.Papers_Perc_Accepted = np.nan

In [69]:
out_row_list = list()

for index, row in df_AcmConferences_raw.iterrows():

    # expansion from the info field
    df_first_level = pd.json_normalize(row['info'])

    for index_first_level, row_first_level in df_first_level.iterrows():
        # check if there's the year
        try:
            row_first_level['year'] 
        except KeyError:
            continue

        if pd.isnull(row_first_level['year']):
            continue # skip row

        # check if is a workshop
        if "student" in row_first_level["yearUnparsed"].lower() or "workshop" in row_first_level["yearUnparsed"].lower():
            continue # skip row

        # filter of row some year speciial cases
        if isinstance(row_first_level['year'], str):
            if row_first_level['year'].__len__() > 4: # special case like "19 Companion", that we need to filter
                continue # skip row

        # normalization of the year format
        year = int(row_first_level['year'])
        if year < 100: # two figures
            year = 2000 + year

        # creation of the support dataframe
        support_dict = dict()
        support_dict['Conf_Acronym'] = row['confAcronym']
        support_dict['Year'] = year

        # removing points or commas 
        submitted = int(str(row_first_level['submitted']).split(".")[0].split(",")[0])
        accepted = int(str(row_first_level['accepted']).split(".")[0].split(",")[0])

        support_dict['Papers_Submitted'] = submitted
        support_dict['Papers_Accepted'] = accepted
        support_dict['Papers_Perc_Accepted'] = int(row_first_level['percAccepted'].split('%')[0])  
        
        # putting the values inside our output list
        out_row_list.append(support_dict)

# conversion of the output list to a dataframe
df_AcmConferences = pd.DataFrame(out_row_list) 


In [70]:
df_AcmConferences
    
    

,Conf_Acronym,Year,Papers_Submitted,Papers_Accepted,Papers_Perc_Accepted
0,DocEng,2019,77,30,39
1,DocEng,2017,71,13,18
2,DocEng,2016,35,11,31
3,DocEng,2015,31,11,35
4,DocEng,2014,41,15,37
...,...,...,...,...,...
4002,GPGPU,2019,15,6,40
4003,GPGPU,2016,23,9,39
4004,DPG,2012,10,6,60
4005,ICAICR,2019,382,49,13


In [ ]:

        
        # extraction of the different dicts
        for items_tuple in df_first_level[df_second_level].items():

            items_dict = items_tuple[1]

            print()

In [ ]:
            # normalization of the year format
            year = items_dict['year']
            if year.__len__() == 2:
                year = '20' + year
            year = int(year)

            # creation of the support dataframe
            support_dict = dict()
            support_dict['Conf_Acronym'] = row['confAcronym']
            support_dict['Year'] = year
            support_dict['Papers_Submitted'] = int(items_dict['submitted'])
            support_dict['Papers_Accepted'] = int(items_dict['accepted'])
            support_dict['Papers_Perc_Accepted'] = int(items_dict['percAccepted'].split('%')[0])  
            
            # putting the values inside our output list
            out_row_list.append(support_dict)

# conversion of the output list to a dataframe
df_AcmConferences = pd.DataFrame(out_row_list) 

## Read and Processing of the CORE CSVs

First of all, we need to define the CSV header.

**Note**: the most of the columns are useless for our purposes, hence I didn't give them a name.

In [43]:
core_csv_header = ["Conference_Series_Full_Name", "ConferenceSeriesNormalizedName", "Source", "Rank", "Has_Data", "Unnamed_1", "Unnamed_2", "Unnamed_3"]

Read and Processing:

In [44]:
df_conference_series_with_core_rank = df_conference_series.copy()

core_all_csvs = glob.glob(path_file_import + "*.csv")

for current_csv_name_full in core_all_csvs:

    current_csv_name = current_csv_name_full.split("/")[-1]

    # Open the current CSV
    print(f'Currently processing CSV: {current_csv_name}')
    df_core_current_csv = pd.read_csv(current_csv_name_full, names=core_csv_header, low_memory=False)

    # Drop of the useless columns: 
    df_core_current_csv.drop(df_core_current_csv.columns.difference(["ConferenceSeriesNormalizedName", "Rank"]), axis=1, inplace=True)

    # Filter of the unranked conferences
    df_core_current_csv = df_core_current_csv[df_core_current_csv["Rank"].str.contains("Unranked") == False]
    df_core_current_csv = df_core_current_csv[df_core_current_csv["Rank"].str.contains("Australasian") == False]
    df_core_current_csv = df_core_current_csv[df_core_current_csv["Rank"].str.contains("National") == False]

    # Drop of the NaN Conference Acrynyms
    df_core_current_csv = df_core_current_csv.dropna(subset={"ConferenceSeriesNormalizedName"})

    # Making the Conference Acronym to Lowecase
    df_core_current_csv.ConferenceSeriesNormalizedName = df_core_current_csv.ConferenceSeriesNormalizedName.str.lower()

    # Drop of the duplicates (there shouldn't be duplicates)
    df_core_current_csv = df_core_current_csv.drop_duplicates(subset="ConferenceSeriesNormalizedName")

    # Reset of the index
    df_core_current_csv = df_core_current_csv.reset_index(drop=True)

    # Rename of the rank column
    df_core_current_csv = df_core_current_csv.rename(columns={'Rank': (current_csv_name.split('.')[0]) + "_Rank"})

    # Left Join with the Distinct Conferences Dataframe
    df_conference_series_with_core_rank = pd.merge(df_conference_series_with_core_rank, df_core_current_csv, on=['ConferenceSeriesNormalizedName'], how='left')

# Column sort
df_conference_series_with_core_rank = df_conference_series_with_core_rank.reindex(sorted(df_conference_series_with_core_rank.columns), axis=1)

Currently processing CSV: CORE_2013.csv
Currently processing CSV: CORE_2017.csv
Currently processing CSV: ERA_2010.csv
Currently processing CSV: CORE_2014.csv
Currently processing CSV: CORE_2018.csv
Currently processing CSV: CORE_2021.csv
Currently processing CSV: CORE_2020.csv
Currently processing CSV: CORE_2008.csv


Let's check the resulting dataframe:

In [45]:
df_conference_series_with_core_rank

,CORE_2008_Rank,CORE_2013_Rank,CORE_2014_Rank,CORE_2017_Rank,CORE_2018_Rank,CORE_2020_Rank,CORE_2021_Rank,ConferenceSeriesNormalizedName,ERA_2010_Rank
0,A,A,A,A,A,A,A,disc,A
1,A,A,A,A,A,A,A,esa,A
2,NaN,C,C,C,C,C,C,enter,C
3,A,B,B,B,B,B,B,dexa,B
4,NaN,C,C,C,C,NaN,NaN,icaisc,C
...,...,...,...,...,...,...,...,...,...
5307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,infinity,NaN
5308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,calculemus,NaN
5309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,agp,NaN
5310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sci,NaN


## Write of the Final CSVs on Disk

Saving the resulting dataframe on disk in CSV format.

In [47]:
# Write of the resulting CSVs on Disk
df_conference_series_with_core_rank.to_csv(path_file_export + 'out_conference_series_with_core_rank.csv')
print(f'Successfully Exported the Joined CSV to {path_file_export}out_conference_series_with_core_rank.csv')

Successfully Exported the Joined CSV to /Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_conference_series_with_core_rank.csv


Check of the Exported CSV to be sure that everything went fine.

In [48]:
# Check of the Exported CSV
df_conference_series_with_core_rank = pd.read_csv(path_file_export + 'out_conference_series_with_core_rank.csv', low_memory=False, index_col=[0])
df_conference_series_with_core_rank

,CORE_2008_Rank,CORE_2013_Rank,CORE_2014_Rank,CORE_2017_Rank,CORE_2018_Rank,CORE_2020_Rank,CORE_2021_Rank,ConferenceSeriesNormalizedName,ERA_2010_Rank
0,A,A,A,A,A,A,A,disc,A
1,A,A,A,A,A,A,A,esa,A
2,NaN,C,C,C,C,C,C,enter,C
3,A,B,B,B,B,B,B,dexa,B
4,NaN,C,C,C,C,NaN,NaN,icaisc,C
...,...,...,...,...,...,...,...,...,...
5307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,infinity,NaN
5308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,calculemus,NaN
5309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,agp,NaN
5310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sci,NaN
